# Homework 2: Decipherment #

### Given code from Default Submission ###

In [32]:
from collections import defaultdict, Counter
from itertools import groupby
import collections
import pprint
import math
import bz2
pp = pprint.PrettyPrinter(width=45, compact=True)

In [2]:
from ngram import LM
lm = LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=False)

Reading language model from data/6-gram-wiki-char.lm.bz2...
Done.


Reading Cipher from file...

In [4]:
def read_file(filename):
    if filename[-4:] == ".bz2":
        with bz2.open(filename, 'rt') as f:
            content = f.read()
            f.close()
    else:
        with open(filename, 'r') as f:
            content = f.read()
            f.close()
    return content

cipher = read_file("data/cipher.txt")
#print(cipher)

In [64]:
def get_statistics(content, cipher=True):
    stats = {}
    content = list(content)
    split_content = [x for x in content if x != '\n' and x!=' ']
    length = len(split_content)
    symbols = set(split_content)
    uniq_sym = len(list(symbols))
    freq = collections.Counter(split_content)
    rel_freq = {}
    for sym, frequency in freq.items():
        rel_freq[sym] = (frequency/length)*100
        
    if cipher:
        stats = {'content':split_content, 'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    else:
        stats = {'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    return stats

## Implementing Baseline Solution ##

First, a list of all possible plaintext symbols.

In [7]:
Ve = [chr(i) for i in range(ord('a'),ord('z')+1)]
print(Ve)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


We must first create a few functions to help us implement a baseline solution using a beam search algorithm.
The functions are derived from the pseudocode provided in assignment 2. Brief descriptions precede each function and will be used together in the function named `beam_search` defined later in the file.    

`find_ext_order` will return a list of ordered ciphertext symbols according to highest frequency.

In [9]:
def find_ext_order(content):
    """
    Finds the ordering of letters for ext_order for highest_frquency
    
    parameters:
    -----------
    content: str
        The entire (ciphertext) string to be transformed into a list of
        symbols to be iterated over in order in the beam search.
    
    returns:
    ----------
    ext_order: list of str
        list of ciphertext symbols returned in descending order of 
        highest frequency
        
    """
    content = list(content)
    split_content = [x for x in content if x != '\n' and x!=' ']
    symbols = set(split_content)
    freq = list(collections.Counter(split_content).items())
    freq.sort(key=lambda x: x[1], reverse=True)
    ext_order = [sym for sym,count in freq]
    return ext_order

`histogram_prune` will keep the best `n_keep` scoring hypotheses and will prune the rest.

In [11]:
def histogram_prune(Ht, n_keep=1):
    """
    keeps the `n_keep` best scoring hypotheses and prunes the rest
    
    parameters:
    -----------
        Ht: list of tup(set, float)
            Hypotheses for matched plaintext and ciphertext symbols (set)
            along with an overall score (float) 
        n_keep: int
            How many hypotheses to keep after pruning lower scoring options
    returns:
    -----------
        best_phis: list of tuples of sets(of tuples)
            returns the n_keep best mappings as well as score
            [({('a','A'), ('b','B')}, 0.8), ({('a','A'), ('g','B')}, 0.6)]
    """
    Ht_sort = sorted(Ht, key=lambda x: x[1], reverse=True)
    best_phis = Ht_sort[:n_keep]
    return best_phis

`winner` Returns the best scoring phi in Hs.

In [21]:
def winner(Hs):
    """Returns the best scoring phi in Hs
    parameters:
    -----------
        Hs: List of tup
            list of tuples of the form (phi, score) where phi is a 
            set of potential mappings and score is the log 
            probability score of such a mapping
    
    returns:
    -----------
        best_phi: set of tuples
            Highest scoring mapping 
    """
    best_phi = [(phi, score) for phi,score in Hs if score == max([s for p,s in Hs])]
    return best_phi

`ext_limits_check` checks the tuples in phi_prime and returns how many times the alphacharacter e, has been mapped to a ciphertext character f in the set phi_prime.

In [22]:
def ext_limits_check(phi_prime, e):
    """
    Checks the tuples in phi_prime and returns how many times the
    alphacharacter e, has been mapped to a ciphertext character f
    in the set phi_prime
    
    parameters:
    -----------
        phi_prime: set of tuples
            set of tuples of the form (e,f) where e is is an 
            alpha-character and f is a ciphertext character
        e: str
            plaintext character e to be counted in phi_prime
    returns:
    -----------
        e_count: int
            the number of times the plaintext character e has
            occured in the set phi_prime in the position (e,f)
    """
    e_s = [e for e,f in phi_prime]
    e_count = e_s.count(e)
    return(e_count)

`g` is a transformation function 'g' returns a sequence in plaintext as well as the corresponding bitstring.

In [23]:
def g(cipher, mappings):
    """
    transformation function 'g' returns a sequence in plaintext
    as well as the corresponding bitstring. To be used in the 
    score_bitstring function from the ngram.py file.
    
    parameters:
    ------------
        cipher: str
            entire ciphertext symbol
        mappings: 
    returns:
    -----------
        seq: str
            partially mapped solution using mappings if the mapping is defined,
            else it returns a dummy symbol '_' 
        bitstring_span: str
            bitstring representation of mapped symbols. 
            'o' if the symbol has already been deciphered 
            '.' if the symbol is yet to be deciphered 
    """
    seq = ""
    bitstring_span = "" # a little confused about this
    match_found = False
    for symbol in cipher:
        for e,f in mappings:
            if symbol == f:
                bitstring_span += "o"
                seq += e #append plain text alphabet
                match_found=True
        if not match_found:
            seq += "_"
            bitstring_span += "."
        else:
            match_found=False
    return seq, bitstring_span

`score` uses the `score_bitstring` functionfrom the language model

In [24]:
def score(seq, bitstring):
    """Score function using the `score_bitstring` function
    from the language model
    
    returns:
    --------
    log probability score of the partially deciphered string
    """
    return lm.score_bitstring(seq, bitstring)

## Implementation of beam search pseudocode! ##

In [29]:
def beam_search(Ve, ext_order, cipher, ext_limits=1, n_keep=1):
    """
    parameters:
    ------------
        Ve: list of str
            list of plaintext candidates
        ext_order: list of str
            list of cipher symbols sorted by their frequencies in the ciphertext
        cipher: str
            cipher text loaded from file
        ext_limits: int
            provides a constraint for the maximum number of cipher symbols
            that can map to each English letter. For a 1:1 substitution cipher
            ext_limits would be 1. For a homophonic cipher it should be 
            greater than 1. 
        n_keep: int
            number of 'best' candidate mappings to keep while traversing the tree
    returns:
    ------------
        winner(Hs): set of tuples
            Best mapping according to the highest score
    """
    cipher = cipher.replace('\n','').strip(' ') # remove newlines and spaces from the cipher
    Vf_size = len(ext_order)
    Hs = list()
    Ht = list()
    cardinality = 0
    Hs.append((set(),0))
    while cardinality < Vf_size:
        f = ext_order[cardinality]
        for phi, s in Hs:
            for e in Ve:
                phi_prime = phi.union({(e,f)})
                e_count = ext_limits_check(phi_prime, e)
                if e_count <= ext_limits:
                    sequence, bitstring = g(cipher, phi_prime)
                    Ht.append((phi_prime, score(sequence, bitstring)))
        cardinality += 1
        Ht = histogram_prune(Ht, n_keep=n_keep)
        if Ht != []:
            Hs = Ht.copy()
        Ht = list()
    return winner(Hs)

First shot at using baseline solution...

In [31]:
cipher = read_file("data/cipher.txt")
ext_order = find_ext_order(cipher)
ext_limits = 3
n_keep = 1000
matches = beam_search(Ve, ext_order, cipher, ext_limits, n_keep)
print("The score is: {}".format(matches[0][1]))

The score is: -607.8314711340004


### Initial Thoughts ###  
We tried various parameters including ext_limits in [1, 2, 3, 4] and n_keep in [1, 100, 1000, 10000] with varying success.   
  
We have also attempted a higher n_keep in order to keep more potential options at each iteration. This clearly was taking a long time and other potential improvements could be made before another attempt that required so much computing power. There also exists room to improve the efficiency of the baseline solution by potentially implementing a priority queue to keep track of scores instead of continuously sorting lists.  
  
There were clearly issues with the baseline solution that could be improved. Upon increasing ext_limits we saw an increase in more frequent letters being mapped multiple times. For example, this caused a decipherment with many 'e's and 't's included. This was clearly not english. This must be further explored in order to choose the best ext_limits. 
  
The ordering of ext_order plays into the problem as well. If more frequent letters are checked first, they tend to have a higher mapping probability and are mapped immediately. This does not allow for less frequent but plausible letters to be considered; they are pruned early on and never considered again. So far, this raises two potential issues: the ordering of ext_order is thus an important consideration as well as our choice for ext_limits.  

This leads us to explore options to potentially improve the following:  
- improving the ordering of ext_order
- potentially implementing a different `ext_limits`
- improving runtime of algorithm through above methods

## Improving Baseline ##

### Improving EXT_ORDER ###

We used two approaches to improve our ext_order beacuse we believe by exploring these ideas we can decrease the overall execution time and improve the quality of our decipherment:

- Firstly, instead of taking the ext_order according to the frequencies of the letters appearing in the cipher, we implemented an ext_order based on the length of the longest deciphered string. For the first character in the ext_order, it chooses the character with the highest frequency in the cipher text as a starting point. After that, it chooses the character next to the longest deciphered substring. So, for example, in case of cipher text: FREEZER, the first character chosen would be "E" (highest frequency). In the second iteration, the next character chosen would be "Z" because it appears next to the longest deciphered susequence, "EE" in this case. This approach performed exceptionally well for 1:1 mapping, but did not do well for the homophonic ciphers.  
   
   
- Secondly, we used the Improved Decipherment of Homophonic Ciphers to weight each cipher symbols to generate a better ext_order based on thier partial decipherment. We used a weighted frequency based approach that orders the symbols based on the maximum number of partial decipherment present. This is intiative because this gives us a higher degree of confidance to pick a better decipherment using previous characters (on every itteration).

We also attempted shuffling the order of ext_order as well as ordering by reverse frequency. These did not show promising results. A final ordering was chosen to be a reverse observed order of the cipher text, explained in more detail in our final solution.

### First implementation of ext_order ###

In [33]:
def len_iter(items):
    """returns the count of longest deciphered sequence (marked by '_') """
    return sum(1 for _ in items)

def choose_next_symbol(cipher_text):
    """chooses the next ciphertext symbol to attempt to decipher based on longest sequence"""
    max_len = max(len_iter(run) for val, run in groupby(cipher_text) if val == "_")
    count = 0
    for idx, symbol in enumerate(cipher_text):
        if symbol == "_":
            count += 1
            if count == max_len:
                return idx + 1
        else:
            count = 0

In [34]:
def beam_search_1(Ve, ext_order, cipher, ext_limits=1, n_keep=1):
    """
        beam_search implementing alternative ext_order
    """
    cipher = cipher.replace('\n','').strip(' ') # remove newlines and spaces from the cipher
    Vf_size = len(ext_order)
    Hs = list()
    Ht = list()
    cardinality = 0
    Hs.append((set(),0))
    vf = cipher
    
    while cardinality < Vf_size:
        if cardinality == 0:
            idx = cardinality
            f = ext_order[idx]
        else:
            idx = choose_next_symbol(vf)
            
            if idx == len(cipher):
                f = ext_order[0]
            else:        
                f = vf[idx]
        
        ext_order.remove(f)    
        vf = vf.replace(f, "_")
        
        for phi, s in Hs:
            for e in Ve:
                phi_prime = phi.union({(e,f)})
                e_count = ext_limits_check(phi_prime, e)
                if e_count <= ext_limits:
                    sequence, bitstring = g(cipher, phi_prime)
                    Ht.append((phi_prime, score(sequence, bitstring)))
        cardinality += 1
        Ht = histogram_prune(Ht, n_keep=n_keep)
        if Ht != []:
            Hs = Ht.copy()
        Ht = list()
    return winner(Hs)

In [36]:
cipher = read_file("data/cipher.txt")
ext_order = find_ext_order(cipher)
ext_limits = 2
n_keep = 100
matches = beam_search_1(Ve, ext_order, cipher, ext_limits, n_keep)
print("The score is: {}".format(matches[0][1]))

The score is: -746.1215271339094


### Second implementation of ext_order ###

In [45]:
def improved_ext_order(ext_order, partial_decipher, processed_list):
    """
    Uses a weighted frequency based approach that orders 
    the symbols based on the maximum number of partial 
    decipherment present
    
    parameters:
    -----------
        ext_order: list 
            (list of) cipher symbols, ordered by weights 
        partial_decipher: str
            partial decipherment of the cipher text generated
        processed_list: list
            contains list of cipher symbols alread processed 
            by the beam search
    returns:
    -----------
        new_ext_order: list
            returns the new ext_order ordered by the weights
    """    
    weights = (1.0,1.0,1.0,1.0,2.0,3.0)
    n_order_occurance_dict = dict()
    symbol_weights = dict()
    for symbol in ext_order:
        n_order_occurance_dict[symbol]={"1_gram_freq":0,
                                        "2_gram_freq":0,
                                        "3_gram_freq":0,
                                        "4_gram_freq":0,
                                        "5_gram_freq":0,
                                        "6_gram_freq":0}
        symbol_weights[symbol]=0
    
    cipher_text_positions = []
    prev_position_value = None
    last_cipher_char_index = -1
    
    current_index = 0
    for char in partial_decipher:

        if char in ext_order:
            cipher_text_positions.append(True)
        else:
            cipher_text_positions.append(False)

        if prev_position_value == False and cipher_text_positions[-1] == True:
        
            no_of_plaintext = current_index - last_cipher_char_index - 1
            
            if no_of_plaintext >= 1:
                n_order_occurance_dict[char]["1_gram_freq"] = n_order_occurance_dict[char]["1_gram_freq"] + 1
            
            if no_of_plaintext >= 2:
                n_order_occurance_dict[char]["2_gram_freq"] = n_order_occurance_dict[char]["2_gram_freq"] + 1
                
            if no_of_plaintext >= 3:
                n_order_occurance_dict[char]["3_gram_freq"] = n_order_occurance_dict[char]["3_gram_freq"] + 1
        
            if no_of_plaintext >= 4:
                n_order_occurance_dict[char]["4_gram_freq"] = n_order_occurance_dict[char]["4_gram_freq"] + 1
            
            if no_of_plaintext >= 5:
                n_order_occurance_dict[char]["5_gram_freq"] = n_order_occurance_dict[char]["5_gram_freq"] + 1
                
            if no_of_plaintext >= 6:
                n_order_occurance_dict[char]["6_gram_freq"] = n_order_occurance_dict[char]["6_gram_freq"] + 1
                    
            #print(char,n_order_occurance_dict[char])
    
        if prev_position_value == True and cipher_text_positions[-1] == False:
            last_cipher_char_index = current_index - 1

            
        prev_position_value = cipher_text_positions[-1]
        current_index = current_index + 1
        
    for symbol,freq_dict in n_order_occurance_dict.items():
        total_weight = 0
#         print("calcualting for {}".format(symbol))
        for order,value in freq_dict.items():
            if order == "1_gram_freq":
                total_weight = total_weight + value*weights[0]
#                 print("First 1st gram weight {}".format(value*weights[0]))
            
            if order == "2_gram_freq":
                total_weight = total_weight + value*weights[1]
#                 print("2 gram weight {}".format(value*weights[1]))
                
            if order == "3_gram_freq":
                total_weight = total_weight + value*weights[2]
#                 print("3 gram weight {}".format(value*weights[2]))
                
            if order == "4_gram_freq":
                total_weight = total_weight + value*weights[3]
#                 print("4 gram weight {}".format(value*weights[3]))
                
            if order == "5_gram_freq":
                total_weight = total_weight + value*weights[4]
#                 print("5 gram weight {}".format(value*weights[4]))
                
            if order == "6_gram_freq":
                total_weight = total_weight + value*weights[5]
#                 print(" 6 gram weight {}".format(value*weights[5]))
                
#             print("total weight weight {}".format(total_weight))
                
       
        symbol_weights[symbol]=total_weight
        
    #assign 0 to symbols in processed list
    for symbol, value in symbol_weights.items():
        if symbol in processed_list:
            symbol_weights[symbol]=0
    
    sorted_by_value = sorted(symbol_weights.items(), key=lambda kv: kv[1])
    sorted_by_value.reverse()

    new_ext_order = []
    
    for val in sorted_by_value:
        new_ext_order.append(val[0])
    
    #print(new_ext_order)
    return new_ext_order

In [51]:
def beam_search_2(Ve, ext_order, cipher, ext_limits=1, n_keep=1):
    Vf_size = len(ext_order)
    Hs = list()
    Ht = list()
    cardinality = 0
    Hs.append((set(),0))
    processed_list=[]
    if ext_limits > Vf_size or ext_limits < 1:
        print("Error: choose a different number for `ext_limits`.\nMust be between 1 and {}".format(Vf_size))
        return
    else:
        while cardinality < Vf_size:
            f = ext_order[0]
            for phi, s in Hs:
                for e in Ve:
                    phi_prime = phi.union({(e,f)})
                    e_count = ext_limits_check(phi_prime, e)
                    if e_count <= ext_limits:
                        sequence, bitstring = g(cipher, phi_prime)
                        Ht.append((phi_prime, score(sequence, bitstring)))
                        ext_order = improved_ext_order(ext_order, sequence, processed_list)
                
            processed_list.append(f)
            cardinality += 1
            # When all plaintext letters have been mapped according to ext_limits
            if(cardinality > ext_limits*len(Ve)):
                break
            Hs = histogram_prune(Ht, n_keep=n_keep)
            Ht = list()
        return winner(Hs)

In [55]:
cipher = read_file("data/cipher.txt")
ext_order = find_ext_order(cipher)
ext_limits = 3
n_keep = 100
matches = beam_search_2(Ve, ext_order, cipher, ext_limits, n_keep)
print("The score is: {}".format(matches[0][1]))

The score is: -643.8926796859997


## Improve EXT_LIMITS ##

We implemented a solution for variable ext_limits, that allows for different ext_limits depending on the plaintext symbol.This results in less freqent letters such as 'x' and 'z' to have ext_limits of 1 and allows for more freqent letters to be mapped more than once. This follows the logic that if a letter occurs less frequently in english, it is less likely to appear in multiple places in the ciphertext and is not likely to have multiple cipher symbols mapped to it. Allowing for more frequent letters to have multiple mappings has a similar problem such that more frequent letters have a higher likelihood than other letters to be mapped more than once.   
The following implementation is based on the observed frequency of letters in the english language. Frequencies are divided into bins, which determine their ext_limits. So we experimented with different ext_limits for different letters accordingly. This gave us a better score and it's usage can be seen in the final beam search algorithm.  

In [57]:
def new_ext_limits():
    plaintxt = read_file("data/default.wiki.txt.bz2")
    plaintxt_desc = get_statistics(plaintxt, cipher=False)

    ext_limits = {}
    for k, v in plaintxt_desc["relative_freq"].items(): 
        if v > 5:
            ext_limits[k] = 2
        elif v > 0.5:
            ext_limits[k] = 4
        else:
            ext_limits[k] = 1
    return ext_limits

## BEST SCORING SOLUTION ##

Implementing an `ext_order` based on an implementation for decipherment in the "An Exact A* Method for Deciphering Letter-Substitution Ciphers" paper suggested an ordering of reverse observed order. This would mean that from right to left from the end of the ciphertext we load the observed letters into `ext_order` exactly. This showed promising results.

This new `ext_order` along with the above mentioned `new_ext_limits` function show us the best results. The following is our final implementation of the beam_search function.

We also saw better results by using the `score_seq` from the language model rather than scoring using the bitstring.

In [60]:
def final_ext_order(content):
    content = list(content)
    split_content = [x for x in content if x != '\n' and x!=' ']
    obs_order = []
    for x in split_content[::-1]:
        if x not in obs_order:
            obs_order.append(x)
    ext_order = obs_order
    return ext_order

In [61]:
def beam_search_final(Ve, ext_order, cipher, ext_limits=1, n_keep=1):
    """
    parameters:
    ------------
        Ve: list of str
            list of plaintext candidates
        ext_order: list of str
            list of cipher symbols sorted by their frequencies in the ciphertext
        cipher: str
            cipher text loaded from file
        ext_limits: int
            provides a constraint for the maximum number of cipher symbols
            that can map to each English letter. For a 1:1 substitution cipher
            ext_limits would be 1. For a homophonic cipher it should be 
            greater than 1. 
        n_keep: int
            number of 'best' candidate mappings to keep while traversing the tree
    returns:
    ------------
        winner(Hs): set of tuples
            Best mapping according to the highest score
    """
    Vf_size = len(ext_order)
    Hs = list()
    Ht = list()
    cardinality = 0
    Hs.append((set(),0))
    while cardinality < Vf_size:
        f = ext_order[cardinality]
        for phi, s in Hs:
            for e in Ve:
                phi_prime = phi.union({(e,f)})
                e_count = ext_limits_check(phi_prime, e)
                if e_count <= ext_limits[e]:
                    sequence, bitstring = g(cipher, phi_prime)
                    score_ = lm.score_seq(sequence)
                    Ht.append((phi_prime, score_))
        cardinality += 1
        Ht = histogram_prune(Ht, n_keep=n_keep)
        if Ht != []:
            Hs = Ht.copy()
        Ht = list()
    return winner(Hs)

In [65]:
cipher = read_file("data/cipher.txt")
cipher = cipher.replace('\n','').strip(' ')
ext_order = final_ext_order(cipher)
n_keep = 200
ext_limits = new_ext_limits()
matches = beam_search_final(Ve, ext_order, cipher, ext_limits, n_keep)
print("The score is: {}".format(matches[0][1]))

The score is: -695.3450931176005


In [68]:
def decipher(mapping, cipher):
    """Given the mapping, translates the cipher"""
    english_text = []
    for symbol in cipher:
        english_text.append(mapping.get(symbol,symbol))
    decipherment = ('').join(english_text)
    return decipherment

In [69]:
mapping = dict((y, x) for x, y in matches[0][0])
decipherment = decipher(mapping,cipher)

In [70]:
filename = "decipher.txt"
with open(filename, "w") as f:
    f.write(decipherment)

In [71]:
print(decipherment)

elifdfallnpbdyodlelrcuyuderitbusycicycaangsbyhkychnyvfellipbfalkbmsonchidcmsbyurlectytrsucebandsothmyvbhyayomptsulbkyllhmfilltosoriacbbnddbsyanesuthhisellicbrpddbhvcoarnuddypleahryhnmcbdraecbubysbmcftockfihitbaylhnhlobrdusaucnhethimhfndvimaynfelllebrlbypicdysmkacdgcmullaihnnydofelldkfilllycosesuultdrtefellpbhbadrumysucyseldcuyboubyfellhyuruuloemofcmbghudsucmlldchipbokuluddtcbysutcarslikedlooseeadsorindaan
